# Day 4 - Numerical Methods

## Agenda
1. finding zeros of functions.
2. scipy.minimize .
3. derivatives and integrals.
4. scipy.interpolate .
5. project.

this section is not meant to be an extensive discussion of numerical methods in python; it will just be a bit of an overview of a handful of the (many) major functions and uses of the `scipy.optimize` library, plus a brief foray into `scipy.interpolate`.

a word of warning: whenever you want to break out any of the numerical methods in `scipy`, __READ THE DOCUMENTATION THOROUGHLY__. there are many very similar functions that on the surface appear to do the same thing, but under the hood and in what they accomplish, can be very different. sometimes, you'll need to switch which one you use because the one you chose just won't converge, or it crashed your computer, or there's a weird error that's not real, or etc etc. it's an art, not a science, so click through the actual documentation and check the 'see also' list at the bottom of each page. 

In [ ]:
# imports for this notebook
import numpy as np
np.set_printoptions(suppress=True) # disable scientific notation
import scipy.optimize, scipy.interpolate, scipy.integrate, scipy.misc
import time
import matplotlib.pyplot as plt

## 1. finding zeros of functions.

one of the most common numerical techniques is finding the zero of a function. sometimes it's finding where our ugly little polynomial crosses the x-axis; sometimes finding the zero of a derivative; but mostly, it's for solving highly nonlinear equations, where we just move all of our terms to one side of the equals sign.

if you have a function in analytic form, the best method to use `scipy.optimize.brentq()`; it uses Richard Brent's algorithm. `brentq` requires two parameters beyond the function, which are the end points of the bracket $[a,b]$ we are finding the zero in. it requires that the signs at $a$ and $b$ are opposite, to guarantee the function crosses zero.

In [ ]:
def f(x):
    return x**3 - 6*x**2 + 11*x - 6

root = scipy.optimize.brentq(f, .5, 1.5)
print(root)

root2 = scipy.optimize.brentq(f, 1.5, 2.5)
print(root2)

root3 = scipy.optimize.brentq(f, 2.5, 10)
print(root3)

root_suprise = scipy.optimize.brentq(f, 0, 10)
print(root_suprise)

root_err = scipy.optimize.brentq(f, 0, 2.5)
print(root_err)

now let's try something like $(x^2-1) \sin(x) = 2x \cos(x)$. we don't know where the root(s) is, so we will plot it to get an idea before using `brentq`.

In [ ]:
# rewrite function to (x^2-1)sinx - 2x cosx = 0
def f(x):
    return (x**2 - 1) * np.sin(x) - 2*x*np.cos(x)

x = np.linspace(0,2,100)
plt.plot(x, f(x))
plt.axhline(0, color='black')
plt.title("Function with unknown root")
plt.show()

we see that there's a root somewhere around x=1.25, so let's use `brentq` to find it:

In [ ]:
root = scipy.optimize.brentq(f, 1, 1.5)
print(root)

__your turn to try!__ use brentq to solve the following:  

(a) $\cos(x) \csc(x) \cot(x) = 6 - \cot^2(x)$  
(b) find the first 3 roots (starting at x=0 -> x=infinity) for $f(x) = \frac{\sin(x)}{x+1}$

a more sophisticated, but considerably more likely to break, method is the `fsolve()` function. it allows us to give an approximation to the root if we would like to choose one:

In [ ]:
def f(x):
    return x**3 - 6*x**2 + 11*x - 6

root = scipy.optimize.fsolve(f, 4)
print(root)

root2 = scipy.optimize.fsolve(f, 0)
print(root2)

`fsolve` is also able to solve nonlinear system of equations, such as this one:

$x \cos(y) - 4 = 0$

$xy - y - 5 = 0$

plotting these is a bit trickier, but here's how to do it:

In [ ]:
def func(x):
    return [x[0] * np.cos(x[1]) - 4,
            x[0] * x[1] - x[1] - 5]

# plotting
x = np.linspace(0,10,100)
y = np.linspace(0,1,20)
X,Y = np.meshgrid(x,y)
F1 = X*np.cos(Y) - 4
F2 = X*Y - Y - 5
plt.contour(X,Y,F1,[0], colors="black")
plt.contour(X,Y,F2,[0], colors="red")
plt.title("system of eqs example")
plt.show()

# finding the root
root = scipy.optimize.fsolve(func, [1, 1])
print(root)

__your turn to try!__ use fsolve to solve the following system of equations, finding all solutions. plot both curves to make sure you find all of them!

$x^2 + y^2 = 26$

$3x^2 + 25y^2 = 100$

hint: call `plt.gca().set_aspect('equal')` before `plt.show()` to get the axes to have the same aspect ratio, if you want your circle to be circular (a cosmetic difference, doesn't affect the math).

## 2. `scipy.optimize.minimize`

`minimize` allows us to solve a constrained equation, which is incredibly common within economics. here's a simple example.

given our equation, $f(x) = (x-1)^2 + (y-2.5)^2 = 0$, we have the following constraints:  

$x - 2y + 2 \geq 0$

$-x - 2y + 6 \geq 0$

$-x + 2y + 2 \geq 0$

both $x$ and $y$ must be greater than 0.

In [ ]:
# recast (x,y) into np.array([x0, x1])
def f(x): 
    (x[0] - 1)**2 + (x[1] - 2.5)**2

# constraints
cons = ({'type': 'ineq', 'fun': lambda x:  x[0] - 2 * x[1] + 2}, 
        {'type': 'ineq', 'fun': lambda x: -x[0] - 2 * x[1] + 6}, 
        {'type': 'ineq', 'fun': lambda x: -x[0] + 2 * x[1] + 2})
bnds = ((0, None), (0, None))

res = scipy.optimize.minimize(f, (2, 1), bounds=bnds, constraints=cons)
print(res)

this is exactly what i mean, i literally pulled this example from the documentation. theoretically this should just be (1.4, 1.7), but go figure. i give up. i hate numerical methods.

__your turn to try!__ solve the following constrained optimization problems (check the documentation to see how to modify constraints and bounds):

(1) $f(x,y) = xy$, constraint: $g(x,y) = x+4y = 240$

(2) find the points on the circle $x^2 + y^2=80$ which are closest and furthest from the point (1,2).

hint: the distance equation to (1,2) is $d = \sqrt{(x-1)^2 + (y-2)^2}$, and $x,y$ are constrained to the circle!

hint 2: maximization can be thought of minimizing the negative version of a function.

## 3. integration and differentiation.

pretty straightforward, we just use the analytic form of the function and the correct bounds. this is not Wolfram Alpha, it will not solve these expressions for us analytically! we only get the values of evaluating the derivative/antiderivative at a particular point/range.

$ \int_{0}^4 x^2 dx $

$ \int_0^\infty e^{-x} dx $

$ \frac{d}{dx} x^3 + x^2 |(x=1) $

In [ ]:
# integration
x2 = lambda x: x**2
y, err = scipy.integrate.quad(x2, 0, 4)
print(y)

invexp = lambda x: np.exp(-x)
y, err = scipy.integrate.quad(invexp, 0, np.inf)
print(y)

In [ ]:
# differentiation
def f(x):
    return x**3 + x**2

der = scipy.misc.derivative(f, 1.0, n=1, dx=1e-6) #n is the order of the derivative
print(der)

what about multiple integrals/derivatives? glad you asked! let's take a look at a double integral first:

we can use the method `dblquad` to solve the following: 

$I = \int_{y=0}^{1/2} \int_{x=0}^{1-2y} xy dx dy$

In [ ]:
area, err = scipy.integrate.dblquad(lambda x, y: x*y, 0, 0.5, lambda x: 0, lambda x: 1-2*x)
print(area)

alternatively, we can use the function `nquad` to perform `n` integrals (in this case 2):

In [ ]:
def f(x, y):
    return x*y

def bounds_y():
    return [0, 0.5]

def bounds_x(y):
    return [0, 1-2*y]

area, err = scipy.integrate.nquad(f, [bounds_x, bounds_y])
print(area)

as another quick example, take a look at solving the eq. below with `nquad`:

$I_n = \int_0^{\infty} \int_1^{\infty} \frac{e^{-xt}}{t^n}dtdx $, for n = 5

In [ ]:
N = 5
def f(t, x):
    return np.exp(-x*t) / t**N

area,err = scipy.integrate.nquad(f, [[1, np.inf],[0, np.inf]])
print(area)

unfortunately, functions for partial derivatives are difficult to come by, so here's one I found for you. source: https://stackoverflow.com/questions/20708038/scipy-misc-derivative-for-multiple-argument-function

In [ ]:
def partial_derivative(func, var=0, point=[]):
    args = point[:]
    def wraps(x):
        args[var] = x
        return func(*args)
    return scipy.misc.derivative(wraps, point[var], dx = 1e-6)

def foo(x, y):
    return x**2 + y**3

print(f"partial x: {partial_derivative(foo, 0, [3,1])}")
print(f"partial y: {partial_derivative(foo, 1, [3,1])}")

## 4. scipy.interpolate

the `interpolate` class is considerably more powerful when it comes to research. this is a python class (like we discussed tuesday) that generates an `interpolate` object.

interpolation is a method of smoothing raw data. there are many, many, methods, but the most common is polynomial interpolation, also called 'splining'. it fits polynomials to subsets of the data, then joins them where they overlap, to form a smooth, continuous, differentiable curve that approximates the original data. naturally, the higher order the polynomial, the better the fit to the data, but having too high an order will overfit your data and ruin any meaningful interpretation you may have had. 

rule of thumb is to use cubic polynomial splines to perform interpolation, but it's a __starting point__ and not a final rule.

let's walk through making a `UnivariateSpline` and some of its useful properties!

In [ ]:
np.random.seed(420) # for reproduceability
x = np.linspace(-3, 3, 50)
y = np.exp(-x**2) + 0.1 * np.random.random(size=50) - .25
plt.plot(x, y, 'ro', ms=5)
plt.axhline(0, linestyle='dashed', color='black')
plt.title("Sample Data")
plt.show()

first, we need to instantiate the `UnivariateSpline` object, which we will then plot on top of our original data.

when creating the support for our spline, we should be very careful not to put in bounds that are outside our original data, as extrapolating requires extra care. while `UnivariateSpline` does allow extrapolation (older versions of this functionality would throw errors), you shouldn't use it to perform analytic work; casual inspection is fine.

In [ ]:
# create the UnivariateSpline object, spl
spl = scipy.interpolate.UnivariateSpline(x, y)

# what support should the spline have?
xs = np.linspace(-3, 3, 1000)

# why we always tune the smoothing factor
# plotting
plt.plot(x, y, 'ro', ms=5)
plt.plot(xs, spl(xs), 'g', lw=3)
plt.title("Sample Data + UnivariateSpline")
plt.show()

...huh.

so why does our spline look like a single parabola, instead of a smooth approximation to our data? well, this is what __oversmoothing__ looks like. we have to tune the smoothing parameter!

n.b. remember our `get()` functions when we were writing our own classes? `UnivariateSpline.set_smoothing_factor()` is pretty much exactly that, except instead of `get`ing a object variable, we are `set`ing it instead! this is what I mean when I say that we shouldn't manipulate object variables directly, but use `get` and `set` functions.

since the smoothing factor is an object variable, once we change it, that smoothing factor doesn't change unless we call `set_smoothing_factor()` again, so once we've chosen it for the spline, we don't need to set it again.

In [ ]:
# why we always tune the smoothing factor
plt.plot(x, y, 'ko', ms=5)
plt.plot(xs, spl(xs), 'g', lw=2) # original spline

spl.set_smoothing_factor(0)
plt.plot(xs, spl(xs), 'black', lw=2)

spl.set_smoothing_factor(.1)
plt.plot(xs, spl(xs), 'r', lw=2)

spl.set_smoothing_factor(.25)
plt.plot(xs, spl(xs), 'orange', lw=2)

spl.set_smoothing_factor(.5)
plt.plot(xs, spl(xs), 'b--', lw=1)

plt.title("Sample Data + UnivariateSpline, multiple smoothing parameters")
plt.legend(["data", "s=1", "s=0", "s=.1", "s=.25", "s=.5"])
plt.show()

i'm going to choose `s=.05` for our spline, pictured here:

In [ ]:
plt.plot(x, y, 'ko', ms=5)
spl.set_smoothing_factor(.05)
plt.plot(xs, spl(xs), 'black', lw=2)

plt.title("Sample Data + UnivariateSpline, smoothed")
plt.show()

now that our spline is ready to go, there's tons of functionality we can take advantage of. funnily enough, we can do everything we've talked about today!

we can evaluate our spline at any point in its support:

In [ ]:
print(spl(0))
print(spl(2))
print(spl(4)) # extrapolation! doesn't throw an error, so be careful

we can find the roots of our spline:

In [ ]:
print(f"roots: {spl.roots()}")

plt.plot(x, y, 'ko', ms=5)
plt.plot(xs, spl(xs), 'black', lw=2)
plt.axvline(spl.roots()[0], color='blue', linestyle='dashed')
plt.axvline(spl.roots()[1], color='blue', linestyle='dashed')
plt.axhline(0, color='green')
plt.show()

we can calculate derivatives at points, and integrals over bounds:

n.b.: be careful with `derivatives()`, if a function is infinitely differentiable, I'm not sure if/when it will stop calculating derivatives. there's probably a cap somewhere, but I honestly don't know.

In [ ]:
print(spl.derivatives(-.5)) # there are 4 derivatives at x=-.5 before our derivative is always 0
print(spl.integral(spl.roots()[0],spl.roots()[1]))

and of course, we can ask it to calculate the residual between the spline and the original data:

In [ ]:
print(spl.get_residual())

__however__, `UnivariateSpline` takes the cake for performing analytic integrals and derivatives!!

In [ ]:
# using the spline to find the derivative
spl_der = spl.derivative()
plt.plot(xs, spl_der(xs))
plt.title("Derivative using UnivariateSpline")
plt.show()

In [ ]:
# and to find the integral
spl_int = spl.antiderivative()
plt.plot(xs, spl_int(xs))
plt.title("Antiderivative using UnivariateSpline")
plt.show()

because calling these functions returns `UnivariateSpline` objects as well, everything we just did we can do with our new `UnivariateSpline`s!!

__NOTE__: some functions above only work with a cubic spline (the default), so taking a derivative or antiderivative converts the splines down/up a degree, respectively. if you want to find the roots, you have to use a 4th order polynomial in the original spline:

In [ ]:
# finding the zeros of the derivative
spl2 = scipy.interpolate.UnivariateSpline(x, y, k=4)
support = np.linspace(-3,3,50)

plt.plot(x,y,'ko')
spl2.set_smoothing_factor(.03) # smoothing factor will change because order changed!
plt.plot(support, spl2(support))
plt.title("4th Order Spline")
plt.show()

In [ ]:
spl2_der = spl2.derivative()
plt.plot(support, spl2_der(support))
plt.axhline(0, color='k')
plt.title("Derivative, 3rd Order Spline")
plt.show()

print(f"zeros of derivative: {spl2.derivative().roots()}")

## 5. project.

here's a messy function for you to try and play around with `UnivariateSpline`! I would like you to find the location and values of the global maxima and minima (of a single period) of the function given to you below. The support for our function is going to be `[0, .02]`

what you should show:
1. rationale for your choice of smoothing parameter
2. plot of the second derivative with `axline` of the global maxima and minima
3. plot of the original noisy data with `axline` of the global maxima and minima, with their corresponding frequency values printed below it (using f strings!).

good luck!

fun fact -- this is actually the sound wave of the A major chord (A - C# - E, A440)!

In [ ]:
np.random.seed(666)

def a_major(x):
    return np.sin(880 * np.pi * x) + np.sin(1100 * np.pi * x) + np.sin(1320 * np.pi * x)

x = np.linspace(0,.02,500)
y = a_major(x) + np.random.random(500)
#plt.plot(x, y, 'bo', ms=3) # if you prefer points
plt.plot(x,y, 'k') # if you prefer lines
plt.title("A Major Chord + Noise")
plt.xlabel("time, s")
plt.ylabel("amplitude")
plt.show()